# Plotting animated GIFs from time series imagery

**What does this notebook do?** 

This notebook demonstrates how to import a time series of DEA cloud-free Landsat imagery from multiple sensors (i.e. Landsat 5, 7 and 8) as an xarray dataset, and then plot the data as an animated time series GIF and a transitioning/fading GIF that compares two timesteps. Animations can be produced as either GIFs or MP4s for any area in Australia using a standard datacube query.

**Required inputs**

This example uses three external functions called `load_clearlandsat`, `animated_timeseries` and `animated_fade`. These functions are available in the Scripts folder of the [dea-notebooks Github repository](https://github.com/GeoscienceAustralia/dea-notebooks/tree/master/Scripts). Note that these functions have been developed by DEA users, not the DEA development team, and so are provided without warranty. If you find an error or bug in the functions, please either create an 'Issue' in the Github repository, or fix it yourself and create a 'Pull' request to contribute the updated function back into the repository (See the repository [README](https://github.com/GeoscienceAustralia/dea-notebooks/blob/master/README.rst) for instructions on creating a Pull request).

**Date:** May 2018

**Author:** Robbi Bishop-Taylor

In [5]:
# Import modules
import datacube 
import sys
import os

# Import external dea-notebooks functions using relative link to Scripts directory
sys.path.append('../Scripts')
import DEADataHandling
import DEAPlotting

# Set up datacube instance
dc = datacube.Datacube(app='Time series animation')

## Set up datacube query
Define the query bounds for datacube extraction using a dict. This should include `x` and `y` limits, potentially a list of `measurements` (i.e. the bands you want to extract like 'red', 'green', 'blue'; this significantly speeds up the import) and a `time` extent. If no `time` is given, the function defaults to all timesteps available to all sensors (e.g. 1987-2018).

In [6]:
from datacube.utils import geometry
from datacube.utils.geometry import CRS

# Set up analysis data query using a buffer around a lat-long point
lat, lon, buffer = -35.712572416, 150.115577708, 5000
x, y = geometry.point(lon, lat, CRS('WGS84')).to_crs(CRS('EPSG:3577')).points[0]
query = {'x': (x - buffer, x + buffer),
         'y': (y - buffer, y + buffer),    
         'measurements': ['swir1', 'nir', 'green', 'red', 'blue'],
         'time': ('2010-01-01', '2016-12-30'),
         'crs': 'EPSG:3577'}


## Extract cloud-free clear Landsat observations from all sensors
Use the `load_clearlandsat` function to load Landsat observations and PQ data for multiple sensors (i.e. ls5, ls7, ls8), and return a single xarray dataset containing only observations that contain greater than a specified proportion of clear pixels. This uses `dask` to only load in the filtered observations, and results in a visually appealing time series of observations that are not affected by cloud!

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import calendar


def animated_doubletimeseries(ds1, ds2, output_path, 
                              bands1=['red', 'green', 'blue'], bands2=['red', 'green', 'blue'], 
                              reflect_stand1=5000, reflect_stand2=5000, title1=None, title2=None,
                              width_pixels=300, interval=50, font_size=25):


    # Define function to convert xarray dataset to list of three band numpy arrays
    def _ds_to_arrraylist(ds, bands, reflect_stand=5000):   

        array_list = []
        for i, timestep in enumerate(ds.time):

                # Select single timestep from the data array
                ds_i = ds.isel(time = i)

                # Create new three band array
                y, x = ds_i[bands[0]].shape
                rawimg = np.zeros((y, x, 3), dtype=np.float32)

                # Add xarray bands into three dimensional numpy array
                for band, colour in enumerate(bands):

                    rawimg[:, :, band] = ds_i[colour].values

                # Stretch contrast using defined reflectance standardisation; defaults to 5000
                img_toshow = (rawimg / reflect_stand).clip(0, 1)
                array_list.append(img_toshow)

        return(array_list)

    # Get height relative to a size of 10 inches width
    width_ratio = float(ds1.sizes['x']) / float(ds1.sizes['y'])
    height = 10 / width_ratio

    # Set up figure
    fig, (ax1, ax2) = plt.subplots(ncols=2) # make figure
    fig.patch.set_facecolor('black')
    fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
    fig.set_size_inches(9.9, height * 0.5, forward=True)
    ax1.axis('off')
    ax2.axis('off')

    # Import source data array
    imagelist1 = _ds_to_arrraylist(ds1, bands=bands1, reflect_stand=reflect_stand1)
    imagelist2 = _ds_to_arrraylist(ds2, bands=bands2, reflect_stand=reflect_stand2)

    # Initialise axesimage objects to be updated during animation
    im1 = ax1.imshow(imagelist1[0])
    im2 = ax2.imshow(imagelist2[0])

    # Initialise annotation objects to be updated during animation
    t1 = ax1.annotate('', xy=(1, 1), xycoords='axes fraction', 
                     xytext=(-5, -5), textcoords='offset points', 
                     horizontalalignment='right', verticalalignment='top', 
                     fontsize=font_size, color = "white", family='monospace')   
    t2 = ax2.annotate('', xy=(1, 1), xycoords='axes fraction', 
                     xytext=(-5, -5), textcoords='offset points', 
                     horizontalalignment='right', verticalalignment='top', 
                     fontsize=font_size, color = "white", family='monospace')  

    # Function to update figure
    def update_figure(frame_i):

        ###################
        # For first panel #
        ###################  

        # Get human-readable date info (e.g. "16 May 1990")
        ts = ds1.time.isel(time=frame_i).dt
        year = ts.year.item()
        month = ts.month.item()
        day = ts.day.item()
        
        # If title:
        if title1:
            date_desc1 = '{}\n{} {} {}'.format(title1, day, calendar.month_abbr[month], year)
        else:
            date_desc1 = '{} {} {}'.format(day, calendar.month_abbr[month], year)

        # Update figure for frame
        im1.set_array(imagelist1[frame_i])
        t1.set_text(date_desc1)

        ####################
        # For second panel #
        #################### 

        # Get human-readable date info (e.g. "16 May 1990")
        ts = ds2.time.isel(time=frame_i).dt
        year = ts.year.item()
        month = ts.month.item()
        day = ts.day.item()
        
        # If title:
        if title2:
            date_desc2 = '{}\n{} {} {}'.format(title2, day, calendar.month_abbr[month], year)
        else:
            date_desc2 = '{} {} {}'.format(day, calendar.month_abbr[month], year)

        # Update figure for frame
        im2.set_array(imagelist2[frame_i])
        t2.set_text(date_desc2) 

        # Return the artists set
        return [im1, im2, t1, t2]

    # Generate and run animation
    ani = animation.FuncAnimation(fig, update_figure, frames=len(ds1.time), interval=interval, blit=True)
    ani.save(output_path, dpi=width_pixels / 9.9, writer='imagemagick')


    
# # Set up query 
# query = {'x': (361850.680639, 370200.0),
#          'y': (-1574202.75509, -1568400.0),   
#          'time': ('2014-12-01', '2015-12-30'),
#          'measurements': ['red', 'green', 'blue'],
#          'crs': 'EPSG:3577'}

# Custom mask that includes only cloudy or cloud shadowed pixels with data for all bands
custom_mask = {'cloud_acca': 'no_cloud', 
               'cloud_fmask': 'no_cloud', 
               'cloud_shadow_acca': 'no_cloud_shadow',
               'cloud_shadow_fmask': 'no_cloud_shadow',
               'contiguous': True}

# Load in data
ds1 = DEADataHandling.load_clearlandsat(dc=dc, query=query, masked_prop=0.83, mask_dict=custom_mask, apply_mask=False)  
ds2 = DEADataHandling.load_clearlandsat(dc=dc, query=query, masked_prop=0.83, mask_dict=custom_mask, apply_mask=True) 

# Animate datasets    
animated_doubletimeseries(ds1=ds1, ds2=ds2, output_path='/home/561/rt1527/Transfer/animation_new.gif',
                          bands1=['red', 'green', 'blue'], bands2=['red', 'green', 'blue'], 
                          reflect_stand1=2500, reflect_stand2=2500, title1='No mask', title2='Cloud mask',
                          width_pixels=800, font_size=12, interval=1000)


TypeError: load_clearlandsat() got an unexpected keyword argument 'mask_dict'

Loading ls8_nbart_albers
Loaded ls8_nbart_albers
Generating mask ls8_pq_albers
Masked ls8_nbart_albers with ls8_pq_albers and filtered terrain
Loading ls8_nbart_albers
Loaded ls8_nbart_albers
Masked ls8_nbart_albers with ls8_pq_albers and filtered terrain


In [187]:
ds.time.isel(time=1).dt

In [117]:
print('Exporting animation to {}'.format(output_path))
input_path = '/g/data/r78/rt1527/mangroves/out_data/*.jpg'
output_path = '/home/561/rt1527/Transfer/animation.mp4'
# !ffmpeg -y -f image2 -pattern_type glob -i '/g/data/r78/rt1527/mangroves/out_data/*.jpg' /home/561/rt1527/Transfer/animation2.wmv
!ffmpeg -y -pattern_type glob -i '/g/data/r78/rt1527/mangroves/out_data/*.jpg' -qscale 3 /home/561/rt1527/Transfer/animation2.wmv
# !convert -antialias -loop 0 -delay 5 $input_path $output_path   

Exporting animation to /home/561/rt1527/Transfer/animation.mp4
ffmpeg version 3.4.2 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 4.8.2 (GCC) 20140120 (Red Hat 4.8.2-15)
  configuration: --prefix=/g/data/v10/public/modules/dea-env/20180405 --disable-doc --enable-shared --enable-static --extra-cflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC -I/g/data/v10/public/modules/dea-env/20180405/include' --extra-cxxflags='=-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-libs='-L/g/data/v10/public/modules/dea-env/20180405/lib -lz' --enable-pic --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --enable-libfreetype --enable-gnutls --enable-libx264
  libavutil      55. 78.100 / 55. 78.100
  libavcodec     57.107.100 / 57.107.100
  libavformat    57. 83.100 / 57. 83.100
  libavdevice    57. 10.100 / 57. 10.100
  libavfilter     6.107.100 /  6.107.100
  libavresample   3.  7.  0 /  3.  7.  0
  libswscale      4.  8.100 /  4.  8.100
  libswresample   

<xarray.Dataset>
Dimensions:    (time: 4, x: 334, y: 233)
Coordinates:
  * y          (y) float64 -1.568e+06 -1.568e+06 -1.568e+06 -1.568e+06 ...
  * x          (x) float64 3.619e+05 3.619e+05 3.619e+05 3.619e+05 3.62e+05 ...
  * time       (time) datetime64[ns] 1990-11-25T00:24:23.500000 ...
Data variables:
    swir1      (time, y, x) float64 272.0 273.0 273.0 273.0 273.0 300.0 ...
    nir        (time, y, x) float64 353.0 395.0 355.0 395.0 395.0 435.0 ...
    green      (time, y, x) float64 1.043e+03 1.048e+03 1.081e+03 1.081e+03 ...
    red        (time, y, x) float64 720.0 782.0 753.0 724.0 754.0 783.0 ...
    data_perc  (time) float64 0.9979 0.9999 0.9983 1.0
Attributes:
    crs:      EPSG:3577